In [2]:
import numpy as np
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
from sklearn.datasets import fetch_20newsgroups

# Cargar el corpus de noticias sin cabeceras, pies de página y citas
newsgroups = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))
newsgroupsdocs = newsgroups.data

In [5]:
# Normalización básica: minúsculas, quitar saltos de línea, tabs y símbolos raros
def normalize_text(text):
    text = text.lower()
    text = text.replace('\n', ' ').replace('\t', ' ')
    text = re.sub(r'[^a-z\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Aplicamos la normalización a todos los documentos
docs = [normalize_text(doc) for doc in newsgroupsdocs]

def remove_stopwords(tokens):
    stopwords = {
        "the", "is", "in", "and", "to", "of", "a", "for", "on", "that", "it",
        "this", "as", "with", "by", "an", "are", "from", "at", "be", "was",
        "or", "which", "we", "can", "has", "have", "will", "not", "if", "but",
        "about", "there", "their", "they", "you", "your", "our", "my", "me",
        "all", "also", "so", "what", "when", "where", "how", "who", "do", "does",
        "did", "would", "could", "should", "may", "might", "must", "been", "being",
        "am", "i"
    }
    filtered_tokens = [word for word in tokens if word not in stopwords]
    return filtered_tokens

# Stemming
def simple_stem(word):
    word = re.sub(r'(ss|ies|s)$', '', word)
    word = re.sub(r'(ing|ed)$', '', word)
    word = re.sub(r'(er|est)$', '', word)
    word = re.sub(r'ly$', '', word)
    return word

def stem_tokens(tokens):
    return [simple_stem(token) for token in tokens]

# Tokenización y pipeline de preprocesamiento
def tokenize(text):
    return re.findall(r'\b[a-z]{2,}\b', text)

def preprocesamiento(text):
    # Aquí el texto ya está normalizado
    tokens = tokenize(text)
    filtered = remove_stopwords(tokens)
    stemmed = stem_tokens(filtered)
    return stemmed

# Procesar todos los documentos
print("\n✓ Aplicando tokenización, eliminación de stopwords y stemming...")
docs_procesados = [preprocesamiento(doc) for doc in docs]

print(f"✓ Documentos procesados: {len(docs_procesados)}")
print(f"\nPrimer documento procesado (10 primeros tokens):")
print(docs_procesados[0][:10])


✓ Aplicando tokenización, eliminación de stopwords y stemming...
✓ Documentos procesados: 18846

Primer documento procesado (10 primeros tokens):
['sure', 'some', 'bash', 'pen', 'fan', 'pretty', 'confus', 'lack', 'any', 'kind']


In [6]:
print("\n" + "="*70)
print("PARTE 1: CÁLCULO DE TF, DF, IDF Y TF-IDF")
print("="*70)

print("\nConstruyendo matriz TF y calculando DF")
print("-" * 50)

# Juntamos los tokens de cada documento otra vez en una sola cadena de texto
docs_procesados_texto = [' '.join(doc) for doc in docs_procesados]

# Matriz de términos (TF) usando CountVectorizer
vectorizer_tf = CountVectorizer()
matriz_tf = vectorizer_tf.fit_transform(docs_procesados_texto)

# Lista de términos del vocabulario
terminos = vectorizer_tf.get_feature_names_out()

# Document Frequency (DF): en cuántos documentos aparece cada término
df_frecuencias = np.array((matriz_tf > 0).sum(axis=0)).flatten()

# DataFrame con info básica de cada término
df_tf_df = pd.DataFrame({
    'termino': terminos,
    'document_frequency': df_frecuencias,
    'total_occurrences': np.array(matriz_tf.sum(axis=0)).flatten()
})

# Ordenamos por DF de mayor a menor
df_tf_df = df_tf_df.sort_values('document_frequency', ascending=False)

print(f"✓ Dimensión de la matriz TF: {matriz_tf.shape}")
print(f"✓ Número de términos distintos: {len(terminos)}")
print(f"✓ Número de documentos: {matriz_tf.shape[0]}")


PARTE 1: CÁLCULO DE TF, DF, IDF Y TF-IDF

Construyendo matriz TF y calculando DF
--------------------------------------------------
✓ Dimensión de la matriz TF: (18846, 77651)
✓ Número de términos distintos: 77651
✓ Número de documentos: 18846


In [11]:
print("\nTop 10 términos por frecuencia de documentos (DF):")
print("-" * 50)
print(df_tf_df.head(10).to_string(index=False))


Top 10 términos por frecuencia de documentos (DF):
--------------------------------------------------
termino  document_frequency  total_occurrences
    one                5435              10979
    any                4774               7846
   like                4511               7270
   some                4333               7865
     no                4321               8307
    out                4213               7300
   know                4104               6466
   just                4090               6193
    oth                3984               7638
    get                3898               6423


In [14]:
print("\n" + "-"*70)
print("Calculando TF-IDF con sklearn")
print("-" * 45)

#TfidfVectorizer sobre los textos ya procesados
tfidf_vectorizer = TfidfVectorizer()
matriz_tfidf = tfidf_vectorizer.fit_transform(docs_procesados_texto)

terminos_tfidf = tfidf_vectorizer.get_feature_names_out()

print(f"Dimensión de la matriz TF-IDF: {matriz_tfidf.shape}")


----------------------------------------------------------------------
Calculando TF-IDF con sklearn
---------------------------------------------
Dimensión de la matriz TF-IDF: (18846, 77651)


In [17]:
print("\n" + "-"*70)
print("Comparación: TF/DF vs TF-IDF")
print("-" * 55)

# Nos quedamos con los primeros 5 documentos para ver valores concretos
tfidf_dense = matriz_tfidf[:5, :].toarray()
tf_dense = matriz_tf[:5, :].toarray()

# Índices de los términos más frecuentes según DF
top_terminos_indices = df_tf_df.index[:30]

comparison_data = []
for idx in top_terminos_indices[:15]:  # mostramos solo 15 filas
    termino = terminos[idx]
    df_val = df_frecuencias[idx]
    tf_promedio = tf_dense[:, idx].mean()
    tfidf_promedio = tfidf_dense[:, idx].mean()

    comparison_data.append({
        'termino': termino,
        'document_frequency': df_val,
        'tf_promedio': round(tf_promedio, 4),
        'tfidf_promedio': round(tfidf_promedio, 6)
    })

df_comparison = pd.DataFrame(comparison_data)
print(df_comparison.to_string(index=False))


----------------------------------------------------------------------
Comparación: TF/DF vs TF-IDF
-------------------------------------------------------
termino  document_frequency  tf_promedio  tfidf_promedio
    one                5435          0.6        0.024080
    any                4774          0.6        0.027679
   like                4511          0.4        0.012728
   some                4333          0.4        0.015059
     no                4321          0.2        0.010032
    out                4213          0.4        0.014786
   know                4104          0.6        0.027094
   just                4090          0.2        0.008790
    oth                3984          0.8        0.033377
    get                3898          0.0        0.000000
    don                3894          0.4        0.013499
   more                3782          0.2        0.010573
     up                3727          0.0        0.000000
     on                3496          0.0     

In [19]:
print("\n" + "="*70)
print("RANKING DE DOCUMENTOS USANDO TF-IDF")
print("="*70)

# Función para obtener el ranking de documentos usando TF-IDF
def ranking_tfidf(consulta, matriz_tfidf, vectorizer, top_n=10):
    # Pasamos la consulta por el mismo preprocesamiento que los docs
    consulta_procesada = preprocesamiento(consulta)
    consulta_texto = ' '.join(consulta_procesada)

    # Representamos la consulta en el espacio TF-IDF
    vector_consulta = vectorizer.transform([consulta_texto])

    # Similaridad coseno entre la consulta y cada documento
    similitudes = cosine_similarity(vector_consulta, matriz_tfidf).flatten()

    # Índices de los docs ordenados de mayor a menor similitud
    indices_ranking = similitudes.argsort()[::-1]

    # Armamos la tabla de resultados
    resultados = []
    for i, idx in enumerate(indices_ranking[:top_n]):
        texto_original = docs[idx]
        muestra = texto_original[:100] + '...' if len(texto_original) > 100 else texto_original

        resultados.append({
            'Rank': i + 1,
            'Doc_ID': idx,
            'Similitud': round(similitudes[idx], 4),
            'Texto_Muestra': muestra
        })

    return pd.DataFrame(resultados)

# Ejemplo rápido de búsqueda
query= "government secret conspiracy "
print(f"Consulta original: '{query}'")
print("-" * 60)

df_ranking = ranking_tfidf(query, matriz_tfidf, tfidf_vectorizer)
print(df_ranking.to_string(index=False))


RANKING DE DOCUMENTOS USANDO TF-IDF
Consulta original: 'government secret conspiracy '
------------------------------------------------------------
 Rank  Doc_ID  Similitud                                                                                           Texto_Muestra
    1   15813     0.4880                                                            government maintaining a secret of some kind
    2    5818     0.2797 jason i ve heard the people who are talking about this dismissed as conspiracy nuts but nobody seems...
    3   17608     0.2468 it will be ironic in the extreme if spector manages to uncover a government conspiracy and cover up ...
    4   11308     0.2210 dear senator congressman president fill in the blank i am writing you to voice my strong opposition ...
    5   15412     0.2198 how about tell everyone what the hell they were doing there in the first place if we knew that we d ...
    6    1282     0.2153 the price you have on the seems very good i too woul